In [ ]:
import pandas as pd

In [ ]:
!pip install PyVCF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for PyVCF: filename=PyVCF-0.6.8-cp37-cp37m-linux_x86_64.whl size=125077 sha256=1c72122049f7db7791a3e7b9a87dee80b671ded44fed2efbf5c227822e88935e
  Stored in directory: /root/.cache/pip/wheels/8a/60/8d/fe98f8401a0bb73b44afbbc454d1d7fe11a0a2081f5b899597
Successfully built PyVCF


In [ ]:
import vcf

In [ ]:
import json

In [ ]:
import time

In [ ]:
import numpy as np

In [ ]:
!pip install h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import h5py

In [ ]:
import numba

In [ ]:
runtimes_1000_genomes = {}

In [ ]:
#/content/Kidd.55AISNP.1kG.vcf
#/content/igsr_samples.tsv
start = time.time()
vcf_reader = vcf.Reader(filename="/content/Kidd.55AISNP.1kG.vcf")
df_1000_genomes = pd.DataFrame(index=vcf_reader.samples)

for variant in vcf_reader:
      df_1000_genomes[variant.ID] = [call.gt_type if call.gt_type is not None else 3 for call in variant.samples]
runtimes_1000_genomes["read_genotypes_samples_var"] = time.time() - start
print(runtimes_1000_genomes)
df_1000_genomes.head()

{'read_genotypes_samples_var': 1.297370433807373}


,rs3737576,rs7554936,rs2814778,rs798443,rs1876482,rs1834619,rs3827760,rs260690,rs6754311,rs10497191,...,rs17642714,rs4471745,rs11652805,rs2042762,rs7226659,rs3916235,rs4891825,rs7251928,rs310644,rs2024566
HG00096,0,1,0,2,0,0,0,1,0,2,...,0,0,1,0,0,2,2,1,1,0
HG00097,0,2,0,2,0,0,0,0,0,2,...,1,0,2,0,0,2,1,0,0,1
HG00099,0,1,0,1,2,1,0,2,0,2,...,0,0,1,0,0,1,1,0,0,1
HG00100,0,2,0,2,0,0,0,2,1,2,...,1,0,2,1,0,2,2,0,0,0
HG00101,0,1,0,2,1,1,0,2,2,2,...,0,0,2,0,0,2,2,1,0,1


In [ ]:
start = time.time()
print(np.unique(df_1000_genomes.values))
runtimes_1000_genomes["genotype_values"] = time.time() - start
print(runtimes_1000_genomes)

[0 1 2]
{'read_genotypes_samples_var': 1.297370433807373, 'genotype_values': 0.006543397903442383}


In [ ]:
start = time.time()
phenotype_data = pd.read_table("/content/igsr_samples.tsv",sep='\t')
phenotype_data_red = phenotype_data[["Sample name", "Superpopulation code", "Superpopulation name"]]
phenotype_data_red = phenotype_data_red.set_index("Sample name")
runtimes_1000_genomes["read_phenotypes_samples"] = time.time() - start
print(runtimes_1000_genomes)
#phenotype_data_red = phenotype_data_red.set_index("Sample name")
phenotype_data_red.head()

{'read_genotypes_samples_var': 1.297370433807373, 'genotype_values': 0.006543397903442383, 'read_phenotypes_samples': 0.018243074417114258}


,Superpopulation code,Superpopulation name
Sample name,,
HG00105,EUR,European Ancestry
HG00112,EUR,European Ancestry
HG00117,EUR,European Ancestry
HG00124,EUR,European Ancestry
HG00129,EUR,European Ancestry


In [ ]:
start = time.time()
phenotype_data_red["Superpopulation name"].value_counts()
runtimes_1000_genomes["phenotype_values"] = time.time() - start
print(runtimes_1000_genomes)

{'read_genotypes_samples_var': 1.297370433807373, 'genotype_values': 0.006543397903442383, 'read_phenotypes_samples': 0.018243074417114258, 'phenotype_values': 0.0016717910766601562}


In [ ]:
phenotype_data_red["Superpopulation name"].value_counts()

African Ancestry                               1413
European Ancestry                               662
South Asian Ancestry                            655
East Asian Ancestry                             618
American Ancestry                               535
East Asia (HGDP)                                193
Central South Asia (HGDP)                       181
Middle East (HGDP)                              153
Europe (HGDP)                                   137
Africa (HGDP)                                    89
West Eurasia (SGDP)                              68
America (HGDP)                                   51
East Asia (SGDP)                                 43
Africa (SGDP)                                    32
South Asia (SGDP)                                29
Central Asia and Siberia (SGDP)                  27
America (SGDP)                                   20
Oceania (SGDP)                                   13
Oceania (HGDP)                                   13
Oceania (SGD

In [ ]:
#df['column name'] = df['column name'].replace(['1st old value','2nd old value',...],'new value')
start = time.time()
phenotype_data_red_cleaned = phenotype_data_red.copy()
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["Africa (HGDP)","Africa (SGDP)","African Ancestry,Africa (SGDP)", "Africa (SGDP),Africa (HGDP)"],"African Ancestry")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["Europe (HGDP)"],"European Ancestry")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["South Asia (SGDP)", "South Asia (SGDP),South Asian Ancestry"],"South Asian Ancestry")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["East Asia (HGDP)", "East Asia (SGDP)", "East Asia (SGDP),East Asian Ancestry", "East Asia (SGDP),East Asia (HGDP)"],"East Asian Ancestry")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["America (HGDP)", "America (SGDP)", "America (SGDP),America (HGDP)"],"American Ancestry")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["Oceania (HGDP)", "Oceania (SGDP),Oceania (HGDP)"],"Oceania (SGDP)")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["Central South Asia (HGDP)"],"Central South Asia")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["Middle East (HGDP)"],"Middle East")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["West Eurasia (SGDP)"],"West Eurasia")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["Central Asia and Siberia (SGDP)"],"Central Asia and Siberia")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["Oceania (SGDP)"],"Oceania")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["European Ancestry,West Eurasia (SGDP)"],"European Ancestry, West Eurasia")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["Middle East (HGDP),Africa (SGDP)"],"Middle East, Africa")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["South Asia (SGDP),Central South Asia (HGDP)"], "South Asia, Central South Asia")
phenotype_data_red_cleaned["Superpopulation name"] = phenotype_data_red_cleaned["Superpopulation name"].replace(["European Ancestry,African Ancestry"],"European Ancestry, African Ancestry")
runtimes_1000_genomes["data_cleaning_phenotypes"] = time.time() - start
print(runtimes_1000_genomes)

{'read_genotypes_samples_var': 1.297370433807373, 'genotype_values': 0.006543397903442383, 'read_phenotypes_samples': 0.018243074417114258, 'phenotype_values': 0.0016717910766601562, 'data_cleaning_phenotypes': 0.01628875732421875}


In [ ]:
print(np.unique(phenotype_data_red_cleaned["Superpopulation name"].values))
print(len(np.unique(phenotype_data_red_cleaned["Superpopulation name"].values)))

['African Ancestry' 'American Ancestry' 'Central Asia and Siberia'
 'Central South Asia' 'East Asian Ancestry' 'European Ancestry'
 'European Ancestry, African Ancestry' 'European Ancestry, West Eurasia'
 'Middle East' 'Middle East, Africa' 'Oceania'
 'South Asia, Central South Asia' 'South Asian Ancestry' 'West Eurasia']
14


In [ ]:
from typing_extensions import runtime
"""
store = pd.HDFStore(path_to_hdf)
store['df1'] = df1
store['df2'] = df2
store.close()"""
#phenotype_data_red_cleaned
#df_1000_genomes
start = time.time()
hdf5_store = pd.HDFStore("/content/1000_genomes_project.h5")
hdf5_store["Genotypes_variants_per_sample"] = df_1000_genomes
hdf5_store["phenotype_data"] = phenotype_data_red_cleaned
hdf5_store.close()
runtimes_1000_genomes["hdf5_file_creation"] = time.time() - start
print(runtimes_1000_genomes)

{'read_genotypes_samples_var': 1.297370433807373, 'genotype_values': 0.006543397903442383, 'read_phenotypes_samples': 0.018243074417114258, 'phenotype_values': 0.0016717910766601562, 'data_cleaning_phenotypes': 0.01628875732421875, 'hdf5_file_creation': 0.028105497360229492}


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Superpopulation code', 'Superpopulation name'], dtype='object')]

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
test_genomes = pd.read_hdf("/content/1000_genomes_project.h5", key="Genotypes_variants_per_sample")
test_genomes.head()

,rs3737576,rs7554936,rs2814778,rs798443,rs1876482,rs1834619,rs3827760,rs260690,rs6754311,rs10497191,...,rs17642714,rs4471745,rs11652805,rs2042762,rs7226659,rs3916235,rs4891825,rs7251928,rs310644,rs2024566
HG00096,0,1,0,2,0,0,0,1,0,2,...,0,0,1,0,0,2,2,1,1,0
HG00097,0,2,0,2,0,0,0,0,0,2,...,1,0,2,0,0,2,1,0,0,1
HG00099,0,1,0,1,2,1,0,2,0,2,...,0,0,1,0,0,1,1,0,0,1
HG00100,0,2,0,2,0,0,0,2,1,2,...,1,0,2,1,0,2,2,0,0,0
HG00101,0,1,0,2,1,1,0,2,2,2,...,0,0,2,0,0,2,2,1,0,1


In [ ]:
test_genomes.empty

False

In [ ]:
with open('runtimes_1000_genomes.json', 'w') as file:
    json.dump(runtimes_1000_genomes, file)

In [ ]:
!pip install bed-reader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.9 MB 32.1 MB/s 


In [ ]:
import numpy as np
from bed_reader import open_bed, sample_file

In [ ]:
!pip install pandas-plink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 9.0 MB/s 
     |████████████████████████████████| 2.5 MB 63.9 MB/s 
     |████████████████████████████████| 298 kB 73.1 MB/s 
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: pytest
    Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4


In [ ]:
from pandas_plink import read_plink1_bin

In [ ]:
G = read_plink1_bin("/content/ADAPTmap_genotypeTOP_20160222_full.bed", "/content/ADAPTmap_genotypeTOP_20160222_full.bim", "/content/ADAPTmap_genotypeTOP_20160222_full.fam", verbose=False)

In [ ]:
print(G)

<xarray.DataArray 'genotype' (sample: 4653, variant: 53347)>
dask.array<transpose, shape=(4653, 53347), dtype=float32, chunksize=(1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * sample   (sample) object 'ET_ABR0001' 'ET_ABR0002' ... 'ET_WYG0046'
  * variant  (variant) <U12 'variant0' 'variant1' ... 'variant53346'
    fid      (sample) object 'ABR' 'ABR' 'ABR' 'ABR' ... 'WYG' 'WYG' 'WYG' 'WYG'
    iid      (sample) object 'ET_ABR0001' 'ET_ABR0002' ... 'ET_WYG0046'
    father   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    mother   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    gender   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    trait    (sample) object '-9' '-9' '-9' '-9' '-9' ... '-9' '-9' '-9' '-9'
    chrom    (variant) object '0' '0' '0' '0' '0' ... '30' '30' '30' '30' '30'
    snp      (variant) object 'snp10134-scaffold1361-15149' ... 'snp32734-sca...
    cm       (variant) float64 0.0 0.0 0.0 0

In [ ]:
print(G.fid)

<xarray.DataArray 'fid' (sample: 4653)>
array(['ABR', 'ABR', 'ABR', ..., 'WYG', 'WYG', 'WYG'], dtype=object)
Coordinates:
  * sample   (sample) object 'ET_ABR0001' 'ET_ABR0002' ... 'ET_WYG0046'
    fid      (sample) object 'ABR' 'ABR' 'ABR' 'ABR' ... 'WYG' 'WYG' 'WYG' 'WYG'
    iid      (sample) object 'ET_ABR0001' 'ET_ABR0002' ... 'ET_WYG0046'
    father   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    mother   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    gender   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    trait    (sample) object '-9' '-9' '-9' '-9' '-9' ... '-9' '-9' '-9' '-9'


In [ ]:
print(G.sample)
#print(list(G.sample))

<xarray.DataArray 'sample' (sample: 4653)>
array(['ET_ABR0001', 'ET_ABR0002', 'ET_ABR0003', ..., 'ET_WYG0044',
       'ET_WYG0045', 'ET_WYG0046'], dtype=object)
Coordinates:
  * sample   (sample) object 'ET_ABR0001' 'ET_ABR0002' ... 'ET_WYG0046'
    fid      (sample) object 'ABR' 'ABR' 'ABR' 'ABR' ... 'WYG' 'WYG' 'WYG' 'WYG'
    iid      (sample) object 'ET_ABR0001' 'ET_ABR0002' ... 'ET_WYG0046'
    father   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    mother   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    gender   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    trait    (sample) object '-9' '-9' '-9' '-9' '-9' ... '-9' '-9' '-9' '-9'


In [ ]:
for el in G.sample:
  print(el)
  break

<xarray.DataArray 'sample' ()>
array('ET_ABR0001', dtype='<U10')
Coordinates:
    sample   <U10 'ET_ABR0001'
    fid      object 'ABR'
    iid      object 'ET_ABR0001'
    father   object '0'
    mother   object '0'
    gender   object '0'
    trait    object '-9'


In [ ]:
#print(G.sel(sample="B003", variant="11_316941526").values)
#variant0' 'variant1' ... 'variant53346
print(G.sel(sample="ET_ABR0009").values)
print(G.sel(sample="ET_ABR0009", variant="variant0").values)
print(G.sel(sample="ET_ABR0009", variant="variant1").values)
print(G.sel(sample="ET_ABR0009", variant="variant2").values)
print(G.sel(sample="ET_ABR0009", variant="variant3").values)
print(G.sel(sample="ET_ABR0009", variant="variant4").values)
print(G.sel(sample="ET_ABR0009", variant="variant5").values)
print(G.sel(sample="ET_ABR0009", variant="variant6").values)
print(G.sel(sample="ET_ABR0009", variant="variant53343").values)
print(G.sel(sample="ET_ABR0009", variant="variant53344").values)
print(G.sel(sample="ET_ABR0009", variant="variant53345").values)
print(G.sel(sample="ET_ABR0009", variant="variant53346").values)

[nan nan  2. ...  2.  2. nan]
nan
nan
2.0
nan
1.0
1.0
2.0
2.0
2.0
2.0
nan


In [ ]:
#print(G.a0.sel(variant="11_316874359").values)
print(G.a0.sel(variant="variant0").values, G.a1.sel(variant="variant0").values)
print(G.a0.sel(variant="variant1").values, G.a1.sel(variant="variant1").values)
print(G.a0.sel(variant="variant2").values, G.a1.sel(variant="variant2").values)
print(G.a0.sel(variant="variant3").values, G.a1.sel(variant="variant3").values)
print(G.a0.sel(variant="variant4").values, G.a1.sel(variant="variant4").values)
print(G.a0.sel(variant="variant5").values, G.a1.sel(variant="variant5").values)
print(G.a0.sel(variant="variant6").values, G.a1.sel(variant="variant6").values)
print(G.a0.sel(variant="variant53343").values, G.a1.sel(variant="variant53343").values)
print(G.a0.sel(variant="variant53344").values, G.a1.sel(variant="variant53344").values)
print(G.a0.sel(variant="variant53345").values, G.a1.sel(variant="variant53345").values)
print(G.a0.sel(variant="variant53346").values, G.a1.sel(variant="variant53346").values)

A G
A G
G A
0 G
G A
G A
G A
G A
G A
A G
G A


In [ ]:
from os.path import join
from pandas_plink import read_plink
from pandas_plink import get_data_folder
(bim, fam, bed) = read_plink(join("/content/ADAPTmap_genotypeTOP_20160222_full.bim",
                                  "/content/ADAPTmap_genotypeTOP_20160222_full.fam",
                                  "/content/ADAPTmap_genotypeTOP_20160222_full.bed"),
                             verbose=False)

In [ ]:
print(bim.head())

  chrom                           snp   cm  pos a0 a1  i
0     0   snp10134-scaffold1361-15149  0.0    0  A  G  0
1     0   snp10135-scaffold1361-44576  0.0    0  A  G  1
2     0   snp10136-scaffold1361-91495  0.0    0  G  A  2
3     0  snp10412-scaffold1372-579082  0.0    0  0  G  3
4     0  snp10413-scaffold1372-610565  0.0    0  G  A  4


In [ ]:
print(type(bim))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
bim.shape

(53347, 7)

In [ ]:
print(fam.head())

   fid         iid father mother gender trait  i
0  ABR  ET_ABR0001      0      0      0    -9  0
1  ABR  ET_ABR0002      0      0      0    -9  1
2  ABR  ET_ABR0003      0      0      0    -9  2
3  ABR  ET_ABR0004      0      0      0    -9  3
4  ABR  ET_ABR0005      0      0      0    -9  4


In [ ]:
df_breedcode_sample = fam[["fid", "iid"]]
df_breedcode_sample.rename(columns={'fid': 'Code breeds'}, inplace=True)
df_breedcode_sample.head()
#print(len(list(fam.iid)))
#print(df_breedcode_sample.fid.values)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Code breeds,iid
0,ABR,ET_ABR0001
1,ABR,ET_ABR0002
2,ABR,ET_ABR0003
3,ABR,ET_ABR0004
4,ABR,ET_ABR0005


In [ ]:
print(len(df_breedcode_sample))

4653


In [ ]:
df_goat_breeds = pd.read_excel('/content/VarGoats data access.xlsx', sheet_name="Sheet1", skiprows=[0])
#df_goat_breeds.columns
df_goat_breeds_codes = df_goat_breeds[["Code breeds", "Breeds"]].drop_duplicates(keep='first')
df_goat_breeds_codes.head()

,Code breeds,Breeds
0,BOE,Boer
4,UNK,Unknown
5,CAS,Cashmere
17,CYL,not relevant
18,DJA,Djallonke


In [ ]:
pd.set_option("display.max_rows", None)
df_goat_breeds_codes["Breeds"].value_counts()
#len(df_goat_breeds_codes)

not relevant                               8
Saanen                                     2
Boer                                       1
Dera Din Panah                             1
Bengal                                     1
Barbari                                    1
Laoshan dairy                              1
Tibetan                                    1
JiningGrey Goat                            1
Thari                                      1
Teddy                                      1
Pateri                                     1
Nachi                                      1
Khurasani                                  1
Kamori                                     1
Kachan                                     1
Damani                                     1
Longlin                                    1
Bari                                       1
Beetal                                     1
Pafuri                                     1
Manica                                     1
Landin    

In [ ]:
df_goat_sample_breed = pd.merge(left=df_breedcode_sample, right=df_goat_breeds_codes, how='left', left_on='Code breeds', right_on='Code breeds')
df_goat_sample_breed[df_goat_sample_breed.duplicated(['iid'], keep=False)]

,Code breeds,iid,Breeds
1648,CAS,AU_CAS0001,Cashmere
1649,CAS,AU_CAS0001,cashmere goat
1650,CAS,AU_CAS0002,Cashmere
1651,CAS,AU_CAS0002,cashmere goat
1652,CAS,AU_CAS0003,Cashmere
1653,CAS,AU_CAS0003,cashmere goat
1654,CAS,AU_CAS0004,Cashmere
1655,CAS,AU_CAS0004,cashmere goat
1656,CAS,AU_CAS0005,Cashmere
1657,CAS,AU_CAS0005,cashmere goat


In [ ]:
len(df_goat_sample_breed)

4724

In [ ]:
df_goat_sample_breed = df_goat_sample_breed[(df_goat_sample_breed["Breeds"] != "cashmere goat") & (df_goat_sample_breed["Breeds"] != "Toggenburger")]
len(df_goat_sample_breed)

4653

In [ ]:
df_goat_sample_breed.set_index("iid", inplace=True)
df_goat_sample_breed.head()

,Code breeds,Breeds
iid,,
ET_ABR0001,ABR,Abergelle
ET_ABR0002,ABR,Abergelle
ET_ABR0003,ABR,Abergelle
ET_ABR0004,ABR,Abergelle
ET_ABR0005,ABR,Abergelle


In [ ]:
df_goat_sample_breed["Breeds"].value_counts()

Angora                413
Boer                  333
Alpine                279
Creole                186
Saanen                171
Landrance             155
Burundi                72
Small East African     55
Abergelle              53
Teddy                  51
Keffa                  49
Cashmere               48
Woyito Guji            46
Rossa Mediterranea     45
Kamori                 42
Gumez                  41
Pateri                 37
Bari                   35
Landin                 33
Girgentana             30
Poitevine              29
Pyrenees               27
Fosses                 26
Matebele               26
Malya                  25
Guera                  25
Peulh                  25
Kachan                 24
Bermeya                24
Sofia                  24
Valdostana             24
Soudanaise             24
Mashona                23
Mubende                23
Galla                  23
Toggenburg             23
Tunisian               23
Sonjo                  22
Targui      

In [ ]:
df_goat_sample_breed.index

Index(['ET_ABR0001', 'ET_ABR0002', 'ET_ABR0003', 'ET_ABR0004', 'ET_ABR0005',
       'ET_ABR0006', 'ET_ABR0007', 'ET_ABR0008', 'ET_ABR0009', 'ET_ABR0010',
       ...
       'ET_WYG0037', 'ET_WYG0038', 'ET_WYG0039', 'ET_WYG0040', 'ET_WYG0041',
       'ET_WYG0042', 'ET_WYG0043', 'ET_WYG0044', 'ET_WYG0045', 'ET_WYG0046'],
      dtype='object', name='iid', length=4653)

In [ ]:
print(bed.compute())

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [ 1.  0.  1. ...  1.  2.  1.]
 ...
 [ 2.  2.  2. ...  2.  2.  2.]
 [ 2.  2.  1. ...  2.  1.  1.]
 [nan nan nan ... nan nan nan]]


In [ ]:
test_bed = bed.compute()

In [ ]:
print(test_bed.shape)

(53347, 4653)


In [ ]:
for el in test_bed:
  print(el)
  break

[nan nan nan ... nan nan nan]


In [ ]:
variants_list = G.variant.values
print(variants_list)
print(type(variants_list))

['variant0' 'variant1' 'variant2' ... 'variant53344' 'variant53345'
 'variant53346']
<class 'numpy.ndarray'>


In [ ]:
df_goat_data = pd.DataFrame(index=df_breedcode_sample.iid, columns=variants_list)
#df_goat_data = pd.DataFrame(index=df_breedcode_sample.iid)
df_goat_data.head()

,variant0,variant1,variant2,variant3,variant4,variant5,variant6,variant7,variant8,variant9,...,variant53337,variant53338,variant53339,variant53340,variant53341,variant53342,variant53343,variant53344,variant53345,variant53346
iid,,,,,,,,,,,,,,,,,,,,,
ET_ABR0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ET_ABR0002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ET_ABR0003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ET_ABR0004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ET_ABR0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
"""for index, row in df_goat_data.iterrows():
  print(index)
  break"""

'for index, row in df_goat_data.iterrows():\n  print(index)\n  break'

In [ ]:
variants = []
sample_list = fam.iid.to_list()
#for index, row in df_goat_data.iterrows():
  #variants = [G.sel(sample=index, variant=variant).values for variant in variants_list]

cross_check = G.sel(sample="ET_ABR0001").values

In [ ]:
samples = fam.iid.to_numpy()

In [ ]:
#for sample in samples:
 # for variant in variants_list:
    #print(sample, variant, df_goat_data.loc[sample][variant])
    #break
  #  df_goat_data.loc[sample][variant] = G.sel(sample=sample, variant=variant).values
  #break
bed[0].shape

(4653,)

In [ ]:
bed_transpose = np.transpose(bed).compute()
print(bed_transpose.shape)
print(bed_transpose)

(4653, 53347)
[[nan nan  1. ...  2.  2. nan]
 [nan nan  0. ...  2.  2. nan]
 [nan nan  1. ...  2.  1. nan]
 ...
 [nan nan  1. ...  2.  2. nan]
 [nan nan  2. ...  2.  1. nan]
 [nan nan  1. ...  2.  1. nan]]


In [ ]:
nr_features_snps = bed_transpose.shape[1]#
excluded_feature_pos = []
for i in range(0, nr_features_snps):
  #row = bed_transpose[:,i]
  values, counts = np.unique(bed_transpose[:,i], return_counts=True)
  counts_sum = counts.sum()
  nan_percentage = counts[-1]/counts_sum
  #print(values, counts, counts_sum, nan_percentage)
  if nan_percentage > 0.05: excluded_feature_pos.append(i)
print(excluded_feature_pos)

[0, 1, 2, 3, 13, 15, 24, 26, 27, 32, 38, 41, 42, 53, 54, 62, 63, 64, 66, 69, 73, 74, 75, 78, 85, 86, 87, 88, 90, 94, 96, 99, 104, 109, 111, 112, 113, 114, 115, 116, 118, 122, 132, 134, 135, 136, 137, 138, 139, 140, 141, 151, 158, 161, 162, 168, 169, 171, 175, 176, 178, 180, 181, 182, 183, 184, 185, 186, 190, 196, 202, 203, 204, 206, 207, 208, 209, 210, 216, 217, 223, 224, 226, 236, 237, 246, 247, 253, 254, 255, 259, 261, 262, 267, 268, 269, 270, 271, 275, 285, 288, 295, 305, 306, 308, 309, 310, 311, 315, 317, 320, 321, 322, 333, 334, 335, 339, 342, 343, 344, 345, 346, 347, 348, 352, 353, 354, 355, 356, 357, 358, 360, 362, 363, 365, 366, 367, 369, 371, 372, 373, 383, 384, 385, 386, 389, 392, 393, 395, 397, 398, 401, 402, 403, 404, 405, 406, 407, 416, 417, 426, 435, 437, 439, 441, 442, 450, 452, 460, 461, 466, 469, 470, 481, 482, 490, 491, 494, 496, 499, 501, 502, 508, 510, 514, 515, 516, 520, 522, 525, 527, 528, 529, 532, 533, 534, 536, 537, 539, 541, 542, 543, 545, 546, 547, 548, 552, 

In [ ]:
print(variants_list)
exluded_variants = []
for pos in excluded_feature_pos:
  exluded_variants.append(variants_list[pos])
print(exluded_variants)
remaining_variants = [var for var in variants_list if var not in exluded_variants]
print(remaining_variants)

['variant0' 'variant1' 'variant2' ... 'variant53344' 'variant53345'
 'variant53346']
['variant0', 'variant1', 'variant2', 'variant3', 'variant13', 'variant15', 'variant24', 'variant26', 'variant27', 'variant32', 'variant38', 'variant41', 'variant42', 'variant53', 'variant54', 'variant62', 'variant63', 'variant64', 'variant66', 'variant69', 'variant73', 'variant74', 'variant75', 'variant78', 'variant85', 'variant86', 'variant87', 'variant88', 'variant90', 'variant94', 'variant96', 'variant99', 'variant104', 'variant109', 'variant111', 'variant112', 'variant113', 'variant114', 'variant115', 'variant116', 'variant118', 'variant122', 'variant132', 'variant134', 'variant135', 'variant136', 'variant137', 'variant138', 'variant139', 'variant140', 'variant141', 'variant151', 'variant158', 'variant161', 'variant162', 'variant168', 'variant169', 'variant171', 'variant175', 'variant176', 'variant178', 'variant180', 'variant181', 'variant182', 'variant183', 'variant184', 'variant185', 'variant186'

In [ ]:
print(len(excluded_feature_pos))

2983


In [ ]:
#bed_copy = np.delete(bed_copy, [0,1], axis=1)
print(bed_transpose.shape)
print(bed_transpose)
bed_copy = bed_transpose.copy()
bed_copy = np.delete(bed_copy, excluded_feature_pos, axis=1)
print(bed_copy.shape)
print(bed_copy)
print(np.unique(bed_copy))

(4653, 53347)
[[nan nan  1. ...  2.  2. nan]
 [nan nan  0. ...  2.  2. nan]
 [nan nan  1. ...  2.  1. nan]
 ...
 [nan nan  1. ...  2.  2. nan]
 [nan nan  2. ...  2.  1. nan]
 [nan nan  1. ...  2.  1. nan]]
(4653, 50364)
[[2. 0. 2. ... 1. 2. 2.]
 [2. 0. 2. ... 2. 2. 2.]
 [2. 0. 2. ... 1. 2. 1.]
 ...
 [2. 0. 2. ... 2. 2. 2.]
 [2. 1. 1. ... 1. 2. 1.]
 [2. 1. 1. ... 2. 2. 1.]]
[ 0.  1.  2. nan]


In [ ]:
#x = x[~numpy.isnan(x)]
nr_features_snps = bed_copy.shape[1]#
for i in range(0, nr_features_snps):
  median_no_nan = np.nanmedian(bed_copy[:,i])
  bed_copy[:,i] = np.round(np.nan_to_num(bed_copy[:,i], nan=median_no_nan))
  #print(np.unique(bed_copy[:,i]), np.median(bed_copy[:,i]))

#print(bed_copy)
print(np.unique(bed_copy))

[0. 1. 2.]


In [ ]:
df_goat_data_red = pd.DataFrame(data=bed_copy, index=df_breedcode_sample.iid, columns=remaining_variants)
df_goat_data_red.head()

,variant4,variant5,variant6,variant7,variant8,variant9,variant10,variant11,variant12,variant14,...,variant53330,variant53331,variant53332,variant53333,variant53336,variant53341,variant53342,variant53343,variant53344,variant53345
iid,,,,,,,,,,,,,,,,,,,,,
ET_ABR0001,2.0,0.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,...,1.0,2.0,0.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0
ET_ABR0002,2.0,0.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,0.0,...,1.0,1.0,0.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0
ET_ABR0003,2.0,0.0,2.0,1.0,2.0,1.0,2.0,0.0,2.0,1.0,...,0.0,2.0,0.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0
ET_ABR0004,2.0,0.0,2.0,2.0,2.0,2.0,1.0,0.0,2.0,1.0,...,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0
ET_ABR0005,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,2.0,0.0,2.0,2.0,2.0,0.0,1.0,2.0,1.0


In [ ]:
print("Genotype values for sample 1 and the first 10 SNP variant")
print(G.sel(sample="ET_ABR0001", variant="variant4").values)
print(G.sel(sample="ET_ABR0001", variant="variant5").values)
print(G.sel(sample="ET_ABR0001", variant="variant6").values)
print(G.sel(sample="ET_ABR0001", variant="variant7").values)
print(G.sel(sample="ET_ABR0001", variant="variant8").values)
print(G.sel(sample="ET_ABR0001", variant="variant9").values)
print(G.sel(sample="ET_ABR0001", variant="variant10").values)
print(G.sel(sample="ET_ABR0001", variant="variant11").values)
print(G.sel(sample="ET_ABR0001", variant="variant12").values)
print(G.sel(sample="ET_ABR0001", variant="variant14").values)

print("Genotype values for sample 2 and the first 10 SNP variant")
print(G.sel(sample="ET_ABR0002", variant="variant4").values)
print(G.sel(sample="ET_ABR0002", variant="variant5").values)
print(G.sel(sample="ET_ABR0002", variant="variant6").values)
print(G.sel(sample="ET_ABR0002", variant="variant7").values)
print(G.sel(sample="ET_ABR0002", variant="variant8").values)
print(G.sel(sample="ET_ABR0002", variant="variant9").values)
print(G.sel(sample="ET_ABR0002", variant="variant10").values)
print(G.sel(sample="ET_ABR0002", variant="variant11").values)
print(G.sel(sample="ET_ABR0002", variant="variant12").values)
print(G.sel(sample="ET_ABR0002", variant="variant14").values)

print("Genotype values for sample 3 and the first 10 SNP variant")
print(G.sel(sample="ET_ABR0003", variant="variant4").values)
print(G.sel(sample="ET_ABR0003", variant="variant5").values)
print(G.sel(sample="ET_ABR0003", variant="variant6").values)
print(G.sel(sample="ET_ABR0003", variant="variant7").values)
print(G.sel(sample="ET_ABR0003", variant="variant8").values)
print(G.sel(sample="ET_ABR0003", variant="variant9").values)
print(G.sel(sample="ET_ABR0003", variant="variant10").values)
print(G.sel(sample="ET_ABR0003", variant="variant11").values)
print(G.sel(sample="ET_ABR0003", variant="variant12").values)
print(G.sel(sample="ET_ABR0003", variant="variant14").values)

print("Genotype values for sample 4 and the first 10 SNP variant")
print(G.sel(sample="ET_ABR0004", variant="variant4").values)
print(G.sel(sample="ET_ABR0004", variant="variant5").values)
print(G.sel(sample="ET_ABR0004", variant="variant6").values)
print(G.sel(sample="ET_ABR0004", variant="variant7").values)
print(G.sel(sample="ET_ABR0004", variant="variant8").values)
print(G.sel(sample="ET_ABR0004", variant="variant9").values)
print(G.sel(sample="ET_ABR0004", variant="variant10").values)
print(G.sel(sample="ET_ABR0004", variant="variant11").values)
print(G.sel(sample="ET_ABR0004", variant="variant12").values)
print(G.sel(sample="ET_ABR0004", variant="variant14").values)

print("Genotype values for sample 5 and the first 10 SNP variant")
print(G.sel(sample="ET_ABR0005", variant="variant4").values)
print(G.sel(sample="ET_ABR0005", variant="variant5").values)
print(G.sel(sample="ET_ABR0005", variant="variant6").values)
print(G.sel(sample="ET_ABR0005", variant="variant7").values)
print(G.sel(sample="ET_ABR0005", variant="variant8").values)
print(G.sel(sample="ET_ABR0005", variant="variant9").values)
print(G.sel(sample="ET_ABR0005", variant="variant10").values)
print(G.sel(sample="ET_ABR0005", variant="variant11").values)
print(G.sel(sample="ET_ABR0005", variant="variant12").values)
print(G.sel(sample="ET_ABR0005", variant="variant14").values)


Genotype values for sample 1 and the first 10 SNP variant
2.0
0.0
2.0
1.0
1.0
2.0
2.0
2.0
1.0
1.0
Genotype values for sample 2 and the first 10 SNP variant
2.0
0.0
2.0
2.0
2.0
1.0
2.0
1.0
2.0
0.0
Genotype values for sample 3 and the first 10 SNP variant
2.0
0.0
2.0
1.0
2.0
1.0
2.0
0.0
2.0
1.0
Genotype values for sample 4 and the first 10 SNP variant
2.0
0.0
2.0
2.0
2.0
2.0
1.0
0.0
2.0
1.0
Genotype values for sample 5 and the first 10 SNP variant
2.0
1.0
2.0
1.0
2.0
2.0
2.0
2.0
2.0
2.0


In [ ]:
hdf5_store = pd.HDFStore("/content/AdaptMap-Goat-Project.h5")
hdf5_store["Genotypes_variants_per_sample"] = df_goat_data_red
hdf5_store["phenotype_data"] = df_goat_sample_breed
hdf5_store.close()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Code breeds', 'Breeds'], dtype='object')]

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#/content/AdaptMap-Goat-Project.h5
!zip -r "/content/AdaptMap-Goat-Project.zip" "/content/AdaptMap-Goat-Project.h5"

  adding: content/AdaptMap-Goat-Project.h5 (deflated 91%)
